In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.externals import joblib

from utils import plot_list

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
rescaled_data = joblib.load('/mnt/ml-team/dsb_2018/kuba/trained_pipelines/rescaled_patched_unet/outputs/rescaler_join')

In [ ]:
meta = pd.read_csv('/mnt/ml-team/dsb_2018/kuba/stage1_metadata.csv')
meta_train = meta[meta['is_train']==1]
raw_images = rescaled_data['X']

In [ ]:
for i, (raw_image, (h,w)) in enumerate(zip(raw_images[0], meta_train[['width','height']].values)):
    print(h,w)
    raw_image = np.array(raw_image)
    plot_list(images=[raw_image])
    if i==10:break